# Chapter 1: Grunnleggende Prompt Struktur- [Lærdom](#lesson)- [Øvelser](#exercises)- [Eksempel Lekeplass](#example-playground)## OppsettKjør følgende oppsett-celle for å laste inn din API-nøkkel og etablere `get_completion` hjelpefunksjon.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---## LærdomAnthropic tilbyr to APIer, the legacy [Text Completions API](https://docs.anthropic.com/claude/reference/complete_post) and the current [Messages API](https://docs.anthropic.com/claude/reference/messages_post). For denne oppstarten, vil vi utelukkende bruke the Messages API.Minimum krever et kall til Claude med Messages API følgende parametere:- `model`: the [API model name](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations) av modellen du har tenkt å kalle- `max_tokens`: maksimalt antall tokens å generere før stopp. Merk at Claude kan stoppe før den når dette maksimum. Denne parameteren spesifiserer kun det absolutte maksimale antallet tokens å generere. Furthermore, this is a *hard* stop, som betyr at det kan få Claude til å stoppe å generere midt i et ord eller midt i en setning.- `messages`: et array av input meldinger. Våre modeller er trent til å operere på vekslende `user` and `assistant` samtaleomgangene. When creating a new `Message`, spesifiserer du de foregående samtaleomgangene med messages parameteren, og modellen genererer deretter the next `Message` i samtalen.  - Hver input melding må være an object with a `role` and `content`. Du kan spesifisere en enkelt `user`-role message, eller du kan inkludere flere `user` and `assistant` messages (de må veksle, hvis så). De first message must always use the user `role`.Dere are also optional parameters, som:- `system`: system prompten - mer om dette nedenfor.  - `temperature`: graden av variabilitet in Claude's svar. For disse leksjonene og øvelser, we have set `temperature` til 0.For en komplett liste av alle API parametere, visit our [API documentation](https://docs.anthropic.com/claude/reference/messages_post).

### EksempelsLa oss se på hvordan Claude svarer på noen riktig-formaterte prompts. For hver av følgende celler, kjør cellen (`shift+enter`), and Claude's svar vil vises under blokken.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Nå la oss se på noen prompts som ikke inkluderer riktig Messages API formatering. For disse feilformaterte promptene, returnerer Messages API en feil.Først har vi et eksempel på et Messages API kall that lacks `role` and `content` fields in the `messages` array.

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

Her er en prompt som feiler å veksle mellom the `user` and `assistant` roles.

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

`user` and `assistant` messages **MUST alternate**, and messages **MUST start with a `user` turn**. You can have multiple `user` & `assistant` pairs in a prompt (as if simulating a multi-turn conversation). You can also put words into a terminal `assistant` message for Claude to continue from where you left off (more on that in later chapters).#### System PromptsYou can also use **system prompts**. A system prompt is a way to **provide context, instructions, and guidelines to Claude** before presenting it with a question or task in the "User" turn. Structurally, system prompts exist separately from the list of `user` & `assistant` messages, and thus belong in a separate `system` parameter (take a look at the structure of the `get_completion` hjelpefunksjon in the [Oppsett](#setup) section of the notebook). Within this tutorial, wherever we might utilize a system prompt, we have provided you a `system` field in your completions function. Should you not want to use a system prompt, simply set the `SYSTEM_PROMPT` variable to an empty string.

#### System Prompt Eksempel

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, som increasing Claude's ability to follow rules and instructions. For more information, visit our documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Eksempel Lekeplass**](#example-playground).

---## Øvelser- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to ThreeUsing proper `user` / `assistant` formatting, edit the `PROMPT` below to get Claude to **count to three.** De output will also indicate whether your solution is correct.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, kjør cellen below!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Exercise 1.2 - System PromptModify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, kjør cellen below!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Congrats!If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---## Eksempel LekeplassDette er et område for deg å eksperimentere fritt med prompten examples vist i denne leksjonen og justere prompts for å se hvordan det kan påvirke Claude's svars.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))